## Ridge and Lasso
At this point we've seen a number of criteria and algorithms for fitting regression models to data. We've seen the simple linear regression using ordinary least squares, and its more general regression of polynomial functions. We've also seen how we can arbitrarily overfit models to data using kernel methods or feature engineering. With all of that, we began to explore other tools to analyze this general problem of overfitting versus underfitting. This included train and test splits, bias and variance, and cross validation.

Now we're going to take a look at another way to tune our models. These methods all modify our mean squared error function that we were optimizing against. The modifications will add a penalty for large coefficient weights in our resulting model. If we think back to our case of feature engineering, we can see how this penalty will help combat our ability to create more accurate models by simply adding additional features.

In general, all of these penalties are known as $L^p norms$.

## $L^p$ norm of x
In order to help account for underfitting and overfitting, we often use what are called $L^p$ norms.   
The **$L^p$ norm of x** is defined as:  

### $||x||_p  =  \big(\sum_{i} x_i^p\big)^\frac{1}{p}$

## 1. Ridge (L2)
One common normalization is called Ridge Regression and uses the $l_2$ norm (also known as the Euclidean norm) as defined above.   
The ridge coefficients minimize a penalized residual sum of squares:    
    $ \sum(\hat{y}-y)^2 + \lambda\bullet w^2$

Write this loss function for performing ridge regression.

In [1]:
import numpy as np

In [2]:
def ridge_loss(y, y_hat, coeff_weights, lam = 0.8):
    rss = np.sum((y_hat-y)**2)
    norm = np.sum(lam * coeff_weights**2)
    l2_err = rss + lam*norm
    return l2_err

## 2. Lasso (L1)
Another common normalization is called Lasso Regression and uses the $l_1$ norm.   
The ridge coefficients minimize a penalized residual sum of squares:    
    $ \sum(\hat{y}-y)^2 + \lambda\bullet |w|$

Write this loss function for performing ridge regression.

In [3]:
def lasso_loss(y, y_hat, coeff_weights, lam = 0.8):
    rss = np.sum((y_hat-y)**2)
    norm = np.sum(lam * np.abs(coeff_weights))
    l1_err = rss + lam*norm
    return l1_err

## 3. Run + Compare your Results
Run a ridge lasso and unpenalized regressions on the dataset below.
While we have practice writing the precursors to a full ridge regression, we'll import the package for now.
Then, answer the following questions:
* Which model do you think created better results overall? 
* Comment on the differences between the coefficients of the resulting models

In [4]:
import pandas as pd

In [5]:
df = pd.read_excel('movie_data_detailed.xlsx')
df.head()

,budget,domgross,title,Response_Json,Year,imdbRating,Metascore,imdbVotes
0,13000000,25682380,21 &amp; Over,0,2008,6.8,48,206513
1,45658735,13414714,Dredd 3D,0,2012,0.0,0,0
2,20000000,53107035,12 Years a Slave,0,2013,8.1,96,537525
3,61000000,75612460,2 Guns,0,2013,6.7,55,173726
4,40000000,95020213,42,0,2013,7.5,62,74170


In [6]:
X = df[['budget', 'imdbRating',
       'Metascore', 'imdbVotes']]
y = df['domgross']

def norm(col):
    minimum = col.min()
    maximum = col.max()
    return (col-maximum)/(maximum-minimum)

In [7]:
from sklearn.linear_model import Lasso, Ridge, LinearRegression
from sklearn.model_selection import train_test_split

#Perform test train spli
X_train , X_test, y_train, y_test = train_test_split(X, y)

ridge_reg = Ridge()
ridge_reg.fit(X_train, y_train)

lasso_reg = Lasso()
lasso_reg.fit(X_train, y_train)

lin_reg = LinearRegression()
lin_reg.fit(X_train, y_train)

def rss(residual_col):
    return sum(residual_col.astype(float).map(lambda x: x**2))

print('Train Error  Ridge Model', rss(y_train - ridge_reg.predict(X_train)))
print('Test Error Ridge Model', rss(y_test - ridge_reg.predict(X_test)))
print('\n')

print('Train Error Lasso Model', rss(y_train - lasso_reg.predict(X_train)))
print('Test Error Lasso Model', rss(y_test - lasso_reg.predict(X_test)))
print('\n')

print('Train Error Unpenalized Linear Model', rss(y_train - lin_reg.predict(X_train)))
print('Test Error Unpenalized  Linear Model', rss(y_test - lin_reg.predict(X_test)))

Train Error  Ridge Model 1.0705617558826253e+17
Test Error Ridge Model 6.095781681848846e+16


Train Error Lasso Model 1.0705355940224341e+17
Test Error Lasso Model 6.105577956427409e+16


Train Error Unpenalized Linear Model 1.0705355940224341e+17
Test Error Unpenalized  Linear Model 6.1055779723063096e+16


In [8]:
#For this dataset there was little difference between the models. Ridge has every so slightly better test performance.

# Altering Alpha

Remember that we can also change our normalization coefficient, alpha, to adjust the strenght of our normalization.
Iterate over the set **np.linspace(start=0.1, stop=2.5, num=13)** in order to find an optimal alpha.

In [9]:
import numpy as np
min_test_error_ridge = []
min_test_error_lasso = []
optimal_ridge_alpha = 0
optimal_lasso_alpha = 0
for iter, a in enumerate(np.linspace(start=0.1, stop=2.5, num=13)):
    ridge_reg = Ridge(alpha=a)
    ridge_reg.fit(X_train, y_train)

    lasso_reg = Lasso(alpha=a)
    lasso_reg.fit(X_train, y_train)

    ridge_train_rss = rss(y_train - ridge_reg.predict(X_train))
    ridge_test_rss = rss(y_test - ridge_reg.predict(X_test))
#     print('Train Error  Ridge Model', ridge_train_rss)
#     print('Test Error Ridge Model', ridge_test_rss)
#     print('\n')
    
    lasso_train_rss = rss(y_train - lasso_reg.predict(X_train))
    lasso_test_rss = rss(y_test - lasso_reg.predict(X_test))
#     print('Train Error Lasso Model', lasso_train_rss)
#     print('Test Error Lasso Model', lasso_test_rss)
#     print('\n')
    
    if iter == 0:
        min_test_error_ridge = ridge_test_rss
        min_test_error_lasso = lasso_test_rss
        optimal_ridge_alpha = a
        optimal_lasso_alpha = a
    if min_test_error_ridge > ridge_test_rss:
        min_test_error_ridge = ridge_test_rss
        optimal_ridge_alpha = a
    if min_test_error_lasso > lasso_test_rss:
        min_test_error_lasso = lasso_test_rss
        optimal_lasso_alpha = a
print('Minimum Ridge Test RSS: {}, Best alpha: {}'.format(min_test_error_ridge, optimal_ridge_alpha))
print('Minimum Lasso Test RSS: {}, Best alpha: {}'.format(min_test_error_lasso, optimal_lasso_alpha))

Minimum Ridge Test RSS: 6.081818824821337e+16, Best alpha: 2.5
Minimum Lasso Test RSS: 6.105577932632072e+16, Best alpha: 2.5
